## Oracle Network 

In [1]:
import pandas as pd
import numpy as np

import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

trdt1=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Genotype\\TrainingGenotype.csv')
trdt=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Geneexpression0\\traingeneexpression.csv',header=None)
tstdt=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Geneexpression0\\testgeneexpression.csv',header=None)

tstdt=np.array(tstdt)
tstdt=np.transpose(tstdt)
tstdt=pd.DataFrame(tstdt)

trdt=np.array(trdt)
trdt=np.transpose(trdt)
trdt=pd.DataFrame(trdt)

results1=trdt1['Phenotype1']
results2=trdt1['Phenotype2']
# convert matrices to pytorch tensors on gpu

trdttensor=torch.from_numpy(trdt.values).type(dtype)
tstdttensor=torch.from_numpy(tstdt.values).type(dtype)
results1tensor=torch.from_numpy(results1.values).type(dtype)
results2tensor=torch.from_numpy(results2.values).type(dtype)

print(trdttensor.shape)
print(tstdttensor.shape)
print(results2tensor.shape)
print(results1tensor.shape)

torch.Size([200, 28395])
torch.Size([30, 28395])
torch.Size([200])
torch.Size([200])


In [3]:
gold=pd.read_csv('C:/Users/Ant Pc/GitHub/PROJECT/DATA/DREAM5_SysGenB2_GoldStandard.csv', sep='\t')
gold.drop(gold.columns[-1],axis=1,inplace=True)

gold2=gold.iloc[1,:]
gold2=gold2.values

dt2=np.transpose(gold2)

gold1=gold.iloc[0,:]
gold1=gold1.values

dt1=np.transpose(gold1)

In [4]:
randmatr=pd.read_csv('C:/Users/Ant Pc/GitHub/PROJECT/Randmatr.csv', header=None).values
class Net1(nn.Module):

    def __init__(self):
        super(Net1, self).__init__()
        
        # an affine operation: y = Wx + b
        
        self.rand=nn.Linear(28395,1000).cuda()
        self.fc1=nn.Linear(1000,3200).cuda()
        self.fc2=nn.Linear(3200,3200).cuda()
        self.fc3=nn.Linear(3200,3200).cuda()
        self.fc6 = nn.Linear(3200,1).cuda()
        
        #set weights as random projection matrix
        self.rand.weight.data = torch.transpose(torch.from_numpy(randmatr),0,1).type(dtype)
        
    def forward(self, x):
        x = self.rand(x).cuda()
        x = F.relu(x).cuda()
        x = F.relu(self.fc1(x)).cuda() 
        x = F.relu(self.fc2(x)).cuda()   
        x = F.relu(self.fc3(x)).cuda()      
        x = self.fc6(x).cuda()
        
        return x

model = Net1()

#use gpu for all computations in model
model.cuda()
print(model)

#trainingloop
import torch.optim as optim

lr = 0.0001
batchsize=20

batches=len(trdttensor)/batchsize

epochs=80

# create your optimizer
optimizer = optim.Adagrad(model.parameters(), lr)


for i in range(epochs):
    
    for j in range(int(batches)):
        
        #forward pass
        out=model(trdttensor[j:j+batchsize,:].type(dtype))

        #compute loss
        criterion = nn.MSELoss()
        loss = criterion(torch.reshape(out,[20]).type(dtype),torch.reshape(results2tensor[j:j+batchsize],[20]).type(dtype)).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #use optimiser to update
        optimizer.step()
    c=nn.L1Loss()
    print(c(torch.reshape(model(tstdttensor),[30]),torch.from_numpy(np.array(dt2)).type(dtype)),i)

Net1(
  (rand): Linear(in_features=28395, out_features=1000, bias=True)
  (fc1): Linear(in_features=1000, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=3200, bias=True)
  (fc3): Linear(in_features=3200, out_features=3200, bias=True)
  (fc6): Linear(in_features=3200, out_features=1, bias=True)
)
tensor(12.7451, device='cuda:0', grad_fn=<L1LossBackward>) 0
tensor(8.5119, device='cuda:0', grad_fn=<L1LossBackward>) 1
tensor(7.2074, device='cuda:0', grad_fn=<L1LossBackward>) 2
tensor(6.3478, device='cuda:0', grad_fn=<L1LossBackward>) 3
tensor(5.7181, device='cuda:0', grad_fn=<L1LossBackward>) 4
tensor(5.2680, device='cuda:0', grad_fn=<L1LossBackward>) 5
tensor(4.8932, device='cuda:0', grad_fn=<L1LossBackward>) 6
tensor(4.5714, device='cuda:0', grad_fn=<L1LossBackward>) 7
tensor(4.3111, device='cuda:0', grad_fn=<L1LossBackward>) 8
tensor(4.6031, device='cuda:0', grad_fn=<L1LossBackward>) 9
tensor(4.1968, device='cuda:0', grad_fn=<L1LossBackward>) 10
tensor(5.20

In [5]:
model(trdttensor[0:20,:])

tensor([[21.0765],
        [26.4424],
        [20.9737],
        [21.9576],
        [17.3590],
        [21.9875],
        [19.9204],
        [19.2679],
        [19.5645],
        [24.7051],
        [22.3846],
        [19.9419],
        [23.6470],
        [35.4223],
        [18.0090],
        [19.9170],
        [20.4232],
        [18.9201],
        [20.7503],
        [18.5378]], device='cuda:0', grad_fn=<AddmmBackward>)

## OGGN

In [6]:
#random input matrix
nsamples=3000
X=np.random.randn(nsamples,6)

Xtensor=torch.from_numpy(X).type(dtype)
print(Xtensor[:,0:30])

tensor([[-0.2831, -0.0210,  0.5410,  0.1419, -0.8578,  0.9457],
        [ 0.0375,  0.0723, -0.6105,  0.1935,  0.1172,  3.4136],
        [-1.6872, -1.4617, -0.4798,  0.9317,  1.0890, -1.1767],
        ...,
        [ 0.6879, -0.9719,  1.3608,  0.6761, -2.2764,  0.5467],
        [-1.5506, -0.3979, -1.5016, -1.1532,  1.2028,  1.3251],
        [-0.6512,  0.1571,  0.5789,  0.6581, -1.5121,  2.9956]],
       device='cuda:0')


In [7]:
def customloss(matr):
    #input is tensor of size(1,28395)
    
    inpu=matr
    
    criterion = nn.MSELoss()
    
    pred=model(matr).type(dtype)
    ideal=45*torch.ones_like(pred).type(dtype)
    loss=criterion(pred,ideal).type(dtype)
    
    return loss

In [8]:
#Neural network to generate geneexpression values
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # an affine operation: y = Wx + b
        self.inp =nn.Linear(6,3200).cuda()
        self.fc1=nn.Linear(3200,3200).cuda()
        self.fc2 = nn.Linear(3200,28395).cuda()
        
    def forward(self, x):
        x=F.relu(self.inp(x)).cuda()
        x = self.fc1(x).cuda()
        x = self.fc2(x).cuda()
        x=torch.abs(x).cuda()
        x[x>10]=x[x>10]/10
        x[x<1]=x[x<1]*100
        x=x.cuda()
        return x

net = Net()
net.cuda()
print(net)

Net(
  (inp): Linear(in_features=6, out_features=3200, bias=True)
  (fc1): Linear(in_features=3200, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=28395, bias=True)
)


In [9]:
#trainingloop
import torch.optim as optim

lr = 0.0001
batchsize=100

batches=nsamples/batchsize

epochs=25

# create your optimizer
optimizer = optim.Adagrad(net.parameters(), lr)

for i in range(epochs):
    
    for j in range(int(batches)):
        
        #forward pass
        out=net(Xtensor[j:j+batchsize,:]).type(dtype)

        #compute loss
        loss = customloss(out).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        optimizer.step()
    print(loss,i)

tensor(62.2916, device='cuda:0', grad_fn=<MseLossBackward>) 0
tensor(28.7720, device='cuda:0', grad_fn=<MseLossBackward>) 1
tensor(15.4282, device='cuda:0', grad_fn=<MseLossBackward>) 2
tensor(9.5249, device='cuda:0', grad_fn=<MseLossBackward>) 3
tensor(6.3478, device='cuda:0', grad_fn=<MseLossBackward>) 4
tensor(4.9480, device='cuda:0', grad_fn=<MseLossBackward>) 5
tensor(3.5314, device='cuda:0', grad_fn=<MseLossBackward>) 6
tensor(3.1139, device='cuda:0', grad_fn=<MseLossBackward>) 7
tensor(2.2698, device='cuda:0', grad_fn=<MseLossBackward>) 8
tensor(2.1668, device='cuda:0', grad_fn=<MseLossBackward>) 9
tensor(2.2770, device='cuda:0', grad_fn=<MseLossBackward>) 10
tensor(1.8379, device='cuda:0', grad_fn=<MseLossBackward>) 11
tensor(1.7276, device='cuda:0', grad_fn=<MseLossBackward>) 12
tensor(1.4037, device='cuda:0', grad_fn=<MseLossBackward>) 13
tensor(1.3237, device='cuda:0', grad_fn=<MseLossBackward>) 14
tensor(1.2920, device='cuda:0', grad_fn=<MseLossBackward>) 15
tensor(1.0821, 

In [10]:
net(Xtensor[0:4,0:10])
n=net(Xtensor)

In [11]:
model(net(Xtensor[0:20,:]))

tensor([[45.3694],
        [43.6997],
        [44.0106],
        [44.6085],
        [44.5296],
        [44.6420],
        [45.6386],
        [43.8766],
        [44.4748],
        [44.1416],
        [44.0472],
        [44.7120],
        [45.5437],
        [44.5248],
        [43.9113],
        [44.2660],
        [45.6405],
        [44.4365],
        [45.3489],
        [45.0733]], device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
result=pd.DataFrame(model(n).detach().cpu().numpy())
print(result.shape)
print(result.max())
print(result.min())
print(result.mean())

(3000, 1)
0    79.809982
dtype: float32
0    22.760593
dtype: float32
0    42.43692
dtype: float32


In [13]:
p=0
preddata=[]
for i in range(len(n)):
    if result.iloc[i,0]>45-0.01:
        if result.iloc[i,0]<45+0.01:
            p=p+1
            preddata.append(n[i].detach().cpu().numpy())

    
print(p)
preddata=np.array(preddata)
preddata.shape

for i in range(len(preddata)):
    predtensori=torch.from_numpy(preddata[i]).type(dtype)
    print(model(predtensori))
    
preddata=pd.DataFrame(preddata)
preddata.to_csv('d45.csv')
print((preddata.mean()).mean())

1
tensor([45.0036], device='cuda:0', grad_fn=<AddBackward0>)
9.443886756896973
